# Version 1 - Simple one agent version


#### Run 1


In [1]:
import os
from pydantic_ai import Agent, RunContext
import logfire
from dotenv import load_dotenv
from tavily import TavilyClient
from pydantic import BaseModel, Field
from rich import print as rprint
from typing import Optional
from src.types import Lead, LeadResults

load_dotenv(dotenv_path="../.env.local")
logfire.configure()
logfire.instrument_pydantic_ai()


tavily_client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))

SYSTEM_PROMPT = """
You are an expert lead research agent specializing in finding high-quality contact information for specific professionals, 
researchers, and business contacts. Your mission is to conduct thorough, systematic research to identify leads that precisely 
match the user's criteria.

## CORE RESEARCH METHODOLOGY

### 1. STRATEGIC SEARCH BREAKDOWN
When given a research query, break it down into these components:
- **WHO**: Specific roles, titles, or professions (e.g., "researchers", "directors", "professors")
- **WHAT**: Industry, field, or specialization (e.g., "Human Nutrition", "Cancer Research", "AI")
- **WHERE**: Geographic location, institution, or organization type (e.g., "Edmonton", "universities", "hospitals")
- **CONTEXT**: Additional qualifiers (e.g., "published authors", "department heads", "recent graduates")

### 2. TOOL USAGE STRATEGY

**Phase 1: Discovery (`browse_web`)**
- Start with broad searches combining WHO + WHAT + WHERE
- Search for: "[profession/role] + [specialization] + [location]"
- Look for: university directories, research institutions, professional associations, conference speakers
- Search variations: Try different keyword combinations, synonyms, and related terms
- Target high-authority sources: .edu domains, government sites, professional organizations

**Phase 2: Site Mapping (`get_website_map`)**
- Use on promising websites found in Phase 1
- Use this before using the `get_website_content` tool
- Focus on: university faculty pages, research department listings, staff directories
- Look for: directory structures, department pages, people/staff sections
- Identify: specific URLs that likely contain detailed contact information

**Phase 3: Deep Extraction (`get_website_content`)**
- Extract detailed content from specific pages identified in Phase 2
- Target: individual profile pages, faculty bios, staff directories, contact pages
- Extract: names, titles, email addresses, phone numbers, research interests, affiliations

### 3. SYSTEMATIC SEARCH PROGRESSION

**Round 1: Institutional Discovery**
1. Search for "[field] researchers [location]" or "[specialty] [location] university"
2. Map promising institutional websites
3. Extract from faculty/staff directory pages

**Round 2: Professional Networks**
1. Search for "[field] association [location]" or "[specialty] conference [location]"
2. Map professional organization websites
3. Extract member directories or speaker lists

**Round 3: Research-Specific Sources**
1. Search for "[research topic] authors [location]" or "publications [specialty] [location]"
2. Map research institution websites
3. Extract from research center or lab pages

**Round 4: Verification & Expansion**
1. Cross-reference findings across multiple sources
2. Search for additional contacts at same institutions
3. Verify current affiliations and contact details

### 4. LEAD QUALITY STANDARDS

**Essential Information (Required):**
- Full name with professional title
- Current institutional affiliation
- At least one direct contact method (email preferred)
- Clear relevance to search criteria

**High-Quality Additions:**
- Multiple contact methods (email + phone)
- Detailed professional summary with relevant expertise
- Current/recent work or research focus
- Professional website or profile links

**Data Accuracy:**
- Prioritize .edu, .org, and official institutional sources
- Look for recently updated information (check page dates)
- Verify contact information appears legitimate (proper email formats)

### 5. SEARCH OPTIMIZATION TECHNIQUES

**Keyword Strategies:**
- Use quotation marks for exact phrases: "cancer research"
- Try both formal and common terms: "faculty" vs "professors" vs "researchers"
- Include location variants: city name, region, institution names
- Use field-specific terminology and acronyms

**Source Prioritization:**
1. University/institutional websites (.edu domains)
2. Professional association directories
3. Research institution websites
4. Government agency listings
5. Professional networking profiles
6. Conference/symposium speaker lists

**Coverage Strategies:**
- Search multiple related institutions in the target area
- Look for both individual researchers and research groups/labs
- Check both current and emeritus faculty
- Include affiliated hospitals, research centers, and institutes

### 6. COMMON PITFALLS TO AVOID
- Don't rely on a single source or search approach
- Don't include outdated or unverifiable contact information
- Don't confuse similar names or mix up affiliations
- Don't include leads that don't clearly match the specified criteria
- Don't stop after finding just a few results - be thorough

### 7. OUTPUT FORMATTING
Structure each lead with:
- Name: Full name with credentials/title
- Email: Direct professional email (verify format)
- Phone: Direct office/professional number when available
- Website: Professional profile or institutional page
- Summary: 2-3 sentences covering current role, specialization, and relevance to search

Find as many leads as possible but avoid bringing leads that a you are not confident about.
"""


class ResearchParams(BaseModel):
    who_query: str = Field(
        description="Specific roles, titles, or professions (e.g., 'researchers', 'directors', 'professors')"
    )
    what_query: str = Field(
        description="Industry, field, or specialization (e.g., 'Human Nutrition', 'Cancer Research', 'AI')"
    )
    where_query: Optional[str] = Field(
        description="Geographic location, institution, or organization type (e.g., 'Edmonton', 'universities', 'hospitals')",
        default=None,
    )
    context_query: Optional[str] = Field(
        description="Additional qualifiers (e.g., 'published authors', 'department heads', 'recent graduates')",
        default=None,
    )


deep_leads_agent = Agent(
    "openai:gpt-4o-mini",
    deps_type=int,
    output_type=LeadResults,
    system_prompt=SYSTEM_PROMPT,
)


@deep_leads_agent.tool
async def browse_web(ctx: RunContext[int], query: str) -> str:
    """browse the web for information"""
    try:
        research_results = tavily_client.search(query, max_results=5)
    except Exception as e:
        print(f"Error browsing the web: {e}")
        return "Error browsing the web"

    return research_results


@deep_leads_agent.tool
async def get_website_map(ctx: RunContext[int], url: str) -> str:
    """get the website map"""
    try:
        website_map = tavily_client.map(url)
    except Exception as e:
        print(f"Error getting the website map: {e}")
        return "Error getting the website map"
    return website_map


@deep_leads_agent.tool
async def get_website_content(ctx: RunContext[int], url: str) -> str:
    """get the website content"""
    try:
        website_content = tavily_client.extract(url)
    except Exception as e:
        print(f"Error getting the website content: {e}")
        return "Error getting the website content"
    return website_content


# Run the agent
params = ResearchParams(
    who_query="professors,",
    what_query="Human Nutrition applied to cancer research",
    where_query="Human Nutrition Research Unit in Edmonton Alberta",
    context_query="professors and researchers who are leading a lab or a team of researchers",
)


def build_final_query(params: ResearchParams) -> str:
    final_query = f"""
    Find me as many leads as possible for the following query:
    
    Who: {params.who_query}
    What is the field of study: {params.what_query}
    Where are they located: {params.where_query}
    Additional context: {params.context_query}
    """
    return final_query


def leads_to_string(leads):
    """
    Converts a list of Lead objects or dicts to a readable string representation.
    """
    if not leads:
        return "No leads found."
    lines = []
    for lead in leads:
        # Support both pydantic Lead objects and dicts
        if not isinstance(lead, Lead):
            raise ValueError(f"Lead is not a Lead object: {lead}")
        data = lead.model_dump()
        line = []
        if data.get("name"):
            line.append(f"Name: {data.get('name', None)}")
        if data.get("title"):
            line.append(f"Title: {data.get('title', None)}")
        if data.get("headline"):
            line.append(f"Headline: {data.get('headline', None)}")
        if data.get("email"):
            line.append(f"Email: {data.get('email', None)}")
        if data.get("phone"):
            line.append(f"Phone: {data.get('phone', None)}")
        if data.get("website"):
            line.append(f"Website: {data.get('website', None)}")
        if data.get("background_summary"):
            line.append(f"Background: {data.get('background_summary', None)}")
        lines.append("\n".join(line))
    return "\n\n".join(lines)


# running a teste example

# RESEARCH_QUERY = build_final_query(params)
# result = await deep_leads_agent.run(RESEARCH_QUERY)
# rprint(result.output)

Logfire project URL: ]8;id=4605;https://logfire-us.pydantic.dev/fmeiraf/maven-rl-course\https://logfire-us.pydantic.dev/fmeiraf/maven-rl-course]8;;\


##### Running Eval


In [ ]:
from deepeval import evaluate, assert_test
from deepeval.test_case import LLMTestCase, LLMTestCaseParams  # type: ignore
from deepeval.metrics import GEval  # type: ignore
from src.evals.human_verified_searches import EDMONTON_HUMAN_NUTRITION_RESEARCH_UNIT
from src.types import EvalParams
from src.evals.lead_comparison import display_leads_comparison


async def test_correctness_with_visual_comparison(eval_params: EvalParams):
    """Enhanced test function with visual lead comparison before evaluation"""
    correctness_metric = GEval(
        name="Correctness",
        model="gpt-4.1-mini",
        criteria="Determine if the 'actual output' contains all the information from the 'expected output'.",
        evaluation_steps=[
            "Check if all the leads on 'actual output' are present in 'expected output'",
            "Check the quality of the leads on 'actual output' compared to 'expected output'",
            "Heavily penalize leads on 'actual output' that are not present in 'expected output'",
        ],
        evaluation_params=[
            LLMTestCaseParams.ACTUAL_OUTPUT,
            LLMTestCaseParams.EXPECTED_OUTPUT,
        ],
        threshold=0.5,
    )

    # getting agent response
    RESEARCH_QUERY = build_final_query(eval_params.query_params)
    result = await deep_leads_agent.run(RESEARCH_QUERY)

    # Visual comparison before evaluation
    print("\n" + "=" * 80)
    print("VISUAL LEAD COMPARISON")
    print("=" * 80)
    display_leads_comparison(result.output.leads, eval_params.expected_results)

    # evaluation
    test_case = LLMTestCase(
        input=RESEARCH_QUERY,
        actual_output=leads_to_string(result.output.leads),
        expected_output=leads_to_string(eval_params.expected_results),
    )

    # in case want assertion and error raising
    # assert_test(test_case, [correctness_metric])

    evaluate(test_cases=[test_case], metrics=[correctness_metric])


# Run the enhanced test
await test_correctness_with_visual_comparison(EDMONTON_HUMAN_NUTRITION_RESEARCH_UNIT)

16:01:16.317 deep_leads_agent run
16:01:16.319   chat gpt-4o-mini
16:01:17.450   running 1 tool
16:01:17.451     running tool: browse_web
16:01:19.206   chat gpt-4o-mini
16:01:21.667   running 3 tools
16:01:21.668     running tool: get_website_map
16:01:23.296     running tool: get_website_map
16:01:23.966     running tool: get_website_map
16:01:26.837   chat gpt-4o-mini
16:01:30.692   running 8 tools
16:01:30.693     running tool: get_website_content
16:01:31.311     running tool: get_website_content
16:01:31.722     running tool: get_website_content
16:01:32.093     running tool: get_website_content
16:01:32.467     running tool: get_website_content
16:01:32.881     running tool: get_website_content
16:01:33.211     running tool: get_website_content
16:01:33.566     running tool: get_website_content
16:01:34.699   chat gpt-4o-mini

VISUAL LEAD COMPARISON


╭──────────────────────────────────────────────────── Summary ────────────────────────────────────────────────────╮
│ LEAD COMPARISON SUMMARY                                                                                         │
│                                                                                                                 │
│ Expected Leads: 2                                                                                               │
│ Actual Leads: 5                                                                                                 │
│ ✓ Matches: 2                                                                                                    │
│ ✗ Missing: 0                                                                                                    │
│ ⚠ Extra: 3                                                                                                      │
│                                                                                                                 │
│ Recall: 100.0%                                                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

                                           ✓ MATCHED LEADS - COMPARISON                                            
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Field                ┃ Actual                               ┃ Expected                             ┃   Match    ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ ──────────────────── │ ──────────────────────────────       │ ──────────────────────────────       │ ────────── │
│ Carla Prado          │                                      │                                      │            │
│ Name                 │ Carla Prado                          │ Carla Prado                          │     ✓      │
│ Title                │ Distinguished University Professor   │ N/A                                  │     ✗      │
│ Email                │ carla.prado@ualberta.ca              │ carla.prado@ualberta.ca              │     ✓      │
│ Phone                │ (780) 492-7934                       │ 780-492-7934                         │     ✗      │
│ Website              │ https://apps.ualberta.ca/directory/… │ https://www.drcarlaprado.com/        │     ✗      │
│ ──────────────────── │ ──────────────────────────────       │ ──────────────────────────────       │ ────────── │
│ Diana Mager          │                                      │                                      │            │
│ Name                 │ Diana Mager                          │ Diana Mager                          │     ✓      │
│ Title                │ Professor                            │ N/A                                  │     ✗      │
│ Email                │ mager@ualberta.ca                    │ mager@ualberta.ca                    │     ✓      │
│ Phone                │ (780) 492-7687                       │ 780-492-7687                         │     ✗      │
│ Website              │ https://apps.ualberta.ca/directory/… │ https://apps.ualberta.ca/directory/… │     ✓      │
└──────────────────────┴──────────────────────────────────────┴──────────────────────────────────────┴────────────┘

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                            ⚠ EXTRA LEADS (3)                                                    │
│ ┏━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓        │
│ ┃ Name            ┃ Title     ┃ Email                ┃ Phone          ┃ Website                        ┃        │
│ ┡━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩        │
│ │ Catherine Field │ Professor │ cjfield@ualberta.ca  │ (780) 492-2597 │ https://apps.ualberta.ca/dire… │        │
│ │ Rene Jacobs     │ Professor │ rjacobs@ualberta.ca  │ (780) 492-2343 │ https://apps.ualberta.ca/dire… │        │
│ │ Vera Mazurak    │ Professor │ vmazurak@ualberta.ca │ (780) 492-8048 │ https://apps.ualberta.ca/dire… │        │
│ └─────────────────┴───────────┴──────────────────────┴────────────────┴────────────────────────────────┘        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────────────────────────────────────────

...

✨ You're running DeepEval's latest Correctness (GEval) Metric! (using gpt-4.1-mini, strict=False, 
async_mode=True)...

Output()

16:01:44.901 Ran evaluate()
16:01:44.905   Ran test case
16:01:44.915 Correctness (GEval)




Metrics Summary

  - ❌ Correctness (GEval) (score: 0.314804719803169, threshold: 0.5, strict: False, evaluation model: gpt-4.1-mini, reason: The actual output includes all expected leads for Carla Prado and Diana Mager but also adds three additional leads not present in the expected output, which is heavily penalized. The quality of leads for Carla Prado is detailed but diverges in website and phone formatting, and the background information is more extensive but not aligned with the expected phrasing. For Diana Mager, the actual output provides relevant background but differs in title and department details. Overall, the presence of extra leads and discrepancies in key details reduce alignment., error: None)

For test case:

  - input: 
    Find me as many leads as possible for the following query:

    Who: professors
    What is the field of study: Human Nutrition
    Where are they located: Human Nutrition Research Unit at University of Alberta
    Additional context: professors 

✓ Tests finished 🎉! Run 'deepeval view' to analyze, debug, and save evaluation results on Confident AI.

In [5]:
from deepeval import evaluate, assert_test
from deepeval.test_case import LLMTestCase, LLMTestCaseParams  # type: ignore
from deepeval.metrics import GEval  # type: ignore
from src.evals.human_verified_searches import EDMONTON_ALES_RESEARCHERS
from src.types import EvalParams
from src.evals.lead_comparison import display_leads_comparison


async def test_correctness_with_visual_comparison(eval_params: EvalParams):
    """Enhanced test function with visual lead comparison before evaluation"""
    correctness_metric = GEval(
        name="Correctness",
        model="gpt-4.1-mini",
        criteria="Determine if the 'actual output' contains all the information from the 'expected output'.",
        evaluation_steps=[
            "Check if all the leads on 'actual output' are present in 'expected output'",
            "Check the quality of the leads on 'actual output' compared to 'expected output'",
            "Heavily penalize leads on 'actual output' that are not present in 'expected output'",
        ],
        evaluation_params=[
            LLMTestCaseParams.ACTUAL_OUTPUT,
            LLMTestCaseParams.EXPECTED_OUTPUT,
        ],
        threshold=0.5,
    )

    # getting agent response
    RESEARCH_QUERY = build_final_query(eval_params.query_params)
    result = await deep_leads_agent.run(RESEARCH_QUERY)

    # Visual comparison before evaluation
    print("\n" + "=" * 80)
    print("VISUAL LEAD COMPARISON")
    print("=" * 80)
    display_leads_comparison(result.output.leads, eval_params.expected_results)

    # evaluation
    test_case = LLMTestCase(
        input=RESEARCH_QUERY,
        actual_output=leads_to_string(result.output.leads),
        expected_output=leads_to_string(eval_params.expected_results),
    )

    evaluate(test_cases=[test_case], metrics=[correctness_metric])


# Run the enhanced test
await test_correctness_with_visual_comparison(EDMONTON_ALES_RESEARCHERS)

16:14:06.910 deep_leads_agent run
16:14:06.911   chat gpt-4o-mini
16:14:08.860   running 1 tool
16:14:08.861     running tool: browse_web
16:14:10.676   chat gpt-4o-mini
16:14:13.989   running 5 tools
16:14:13.989     running tool: get_website_map
16:14:14.764     running tool: get_website_map
Error getting the website map: Request timed out after 60 seconds.
16:15:14.900     running tool: get_website_map
16:15:15.728     running tool: get_website_map
16:15:16.953     running tool: get_website_map
16:15:17.819   chat gpt-4o-mini
16:15:22.645   running 3 tools
16:15:22.645     running tool: get_website_content
16:15:23.052     running tool: get_website_content
16:15:23.458     running tool: get_website_content
16:15:23.803   chat gpt-4o-mini

VISUAL LEAD COMPARISON


╭──────────────────────────────────────────────────── Summary ────────────────────────────────────────────────────╮
│ LEAD COMPARISON SUMMARY                                                                                         │
│                                                                                                                 │
│ Expected Leads: 53                                                                                              │
│ Actual Leads: 10                                                                                                │
│ ✓ Matches: 8                                                                                                    │
│ ✗ Missing: 45                                                                                                   │
│ ⚠ Extra: 2                                                                                                      │
│                                                                                                                 │
│ Recall: 15.1%                                                                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

                                           ✓ MATCHED LEADS - COMPARISON                                            
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Field                ┃ Actual                               ┃ Expected                             ┃   Match    ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ ──────────────────── │ ──────────────────────────────       │ ──────────────────────────────       │ ────────── │
│ Spencer Proctor      │                                      │                                      │            │
│ Name                 │ Spencer Proctor                      │ Spencer Proctor                      │     ✓      │
│ Title                │ Professor & Chair                    │ Professor                            │     ✗      │
│ Email                │ spencer.proctor@ualberta.ca          │ proctor@ualberta.ca                  │     ✗      │
│ Phone                │ 780-492-4672                         │ (780) 492-4672                       │     ✗      │
│ Website              │ https://apps.ualberta.ca/directory/… │ https://mcvd.ualberta.ca/            │     ✗      │
│ ──────────────────── │ ──────────────────────────────       │ ──────────────────────────────       │ ────────── │
│ Burim Ametaj         │                                      │                                      │            │
│ Name                 │ Burim Ametaj                         │ Burim Ametaj                         │     ✓      │
│ Title                │ Professor                            │ Professor                            │     ✓      │
│ Email                │ burim.ametaj@ualberta.ca             │ bametaj@ualberta.ca                  │     ✗      │
│ Phone                │ 780-492-9841                         │ (780) 492-9841                       │     ✗      │
│ Website              │ https://apps.ualberta.ca/directory/… │ https://www.ualberta.ca/             │     ✗      │
│ ──────────────────── │ ──────────────────────────────       │ ──────────────────────────────       │ ────────── │
│ Daniel Barreda       │                                      │                                      │            │
│ Name                 │ Daniel Barreda                       │ Daniel Barreda                       │     ✓      │
│ Title                │ Professor (Joint with BioSci)        │ Professor                            │     ✗      │
│ Email                │ dan.barreda@ualberta.ca              │ d.barreda@ualberta.ca                │     ✗      │
│ Phone                │ 780-492-0375                         │ 780-492-0375                         │     ✓      │
│ Website              │ https://apps.ualberta.ca/directory/… │ https://apps.ualberta.ca/directory/… │     ✓      │
│ ──────────────────── │ ──────────────────────────────       │ ──────────────────────────────       │ ────────── │
│ Rene Jacobs          │                                      │                                      │            │
│ Name                 │ Rene Jacobs                          │ Rene Jacobs                          │     ✓      │
│ Title                │ Professor                            │ Professor                            │     ✓      │
│ Email                │ rjacobs@ualberta.ca                  │ rjacobs@ualberta.ca                  │     ✓      │
│ Phone                │ 780-492-2343                         │ (780) 492-2343                       │     ✗      │
│ Website              │ https://apps.ualberta.ca/directory/… │ https://www.ualberta.ca/             │     ✗      │
│ ──────────────────── │ ──────────────────────────────       │ ──────────────────────────────       │ ────────── │
│ Carla Prado          │                                      │                                      │            │
│ Name                 │ Carla Prado                    

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                              ✗ MISSING LEADS (45)                                               │
│ ┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓ │
│ ┃ Name                 ┃ Title                ┃ Email                ┃ Phone           ┃ Website              ┃ │
│ ┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩ │
│ │ David Bressler       │ Professor            │ dbressle@ualberta.ca │ (780) 492-4986  │ http://bresslerlab.… │ │
│ │ Erick R. da Silva    │ Assistant Professor  │ ericksantos@ualbert… │ N/A             │ https://www.ualbert… │ │
│ │ Santos               │                      │                      │                 │                      │ │
│ │ Thava Vasanthan      │ Professor            │ tv3@ualberta.ca      │ (780) 492-2898  │ https://www.ualbert… │ │
│ │ Marleny Aranda       │ Professor            │ marleny@ualberta.ca  │ (780) 492-8018  │ https://www.ualbert… │ │
│ │ Saldana              │                      │                      │                 │                      │ │
│ │ Gleise Medeiros da   │ Assistant Professor  │ gleise.silva@ualber… │ (780) 492-0237  │ https://www.ualbert… │ │
│ │ Silva                │                      │                      │                 │                      │ │
│ │ Roopesh Syamaladevi  │ Associate Professor  │ roopeshms@ualberta.… │ (780) 492-8413  │ https://foodsafetye… │ │
│ │ Hiwot Haileslassie   │ Associate Lecturer   │ hhailesl@ualberta.ca │ (780) 492-7742  │ https://www.ualbert… │ │
│ │ Rudolph              │ Assistant Professor  │ freduaag@ualberta.ca │ (780) 492-3685  │ https://scholar.goo… │ │
│ │ Fredua-Agyeman       │                      │                      │                 │                      │ │
│ │ Aman Ullah           │ Professor            │ ullah2@ualberta.ca   │ 780-492-4845    │ https://apps.ualber… │ │
│ │ Michael Dyck         │ Professor            │ mkdyck@ualberta.ca   │ 780-492-0047    │ https://www.ualbert… │ │
│ │ Malinda              │ Assistant Professor  │ thilakar@ualberta.ca │ (780) 492-9966  │ https://plantmicrob… │ │
│ │ Thilakarathna        │                      │                      │                 │                      │ │
│ │ Wendy Wismer         │ Associate Professor  │ wwismer@ualberta.ca  │ 780-492-2923    │ https://gradpositio… │ │
│ │ Carolyn Fitzsimmons  │ PhD                  │ cfitzsim@ualberta.ca │ 780-248-1700    │ https://apps.ualber… │ │
│ │ Brasathe Jeganathan  │ Assistant Lecturer   │ jeganath@ualberta.ca │ 780-492-3829    │ https://apps.ualber… │ │
│ │ Linda Gorim          │ Assistant Professor  │ gorim@ualberta.ca    │ 780-492-8814    │ https://apps.ualber… │ │
│ │                      │ & WGRF Chair in      │                      │                 │                      │ │
│ │                      │ Cropping Systems     │                      │                 │                      │ │
│ │ Ning Xiang           │ Assistant Professor  │ nxiang@ualberta.ca   │ 780-492-4610    │ http://www.afns.ual… │ │
│ │ Stephen Strelkov     │ Professor            │ strelkov@ualberta.ca │ 780-492-1969    │ https://apps.ualber… │ │
│ │ Jianping Wu          │ Professor            │ jwu3@ualberta.ca     │ (780) 492-6885  │ https://apps.ualber… │ │
│ │ Guanqun (Gavin) Chen │ Associate Professor  │ gc24@ualberta.ca     │ 780-492-3148    │ https://plb.ualbert… │ │
│ │ Anne Laarman         │ Assistant Professor  │ alaarman@ualberta.ca │ (780) 492-8228  │ https://scholar.goo… │ │
│ │ Catherine Field      │ Professor            │ catherine.field@ual… │ (780) 492-2597  │ https://orcid.org/0… │ │
│ │ Richard Uwiera       │ Professor            │ ruwiera@ualberta.ca  │ (780) 492-0107  │ https://apps.ualber… │ │
│ │ Sheau-Fang Hwang     │ Professor            │ sh20@u

────────────────────────────────────────────────────────────────────────────────

...

✨ You're running DeepEval's latest Correctness (GEval) Metric! (using gpt-4.1-mini, strict=False, 
async_mode=True)...

Output()

16:15:40.067 Ran evaluate()
16:15:40.070   Ran test case
16:15:40.072 Correctness (GEval)




Metrics Summary

  - ❌ Correctness (GEval) (score: 0.2, threshold: 0.5, strict: False, evaluation model: gpt-4.1-mini, reason: The actual output contains many leads not present in the expected output, violating step 3 and heavily penalizing the response. Only a few names overlap (e.g., Spencer Proctor, Burim Ametaj, Daniel Barreda, Rene Jacobs, Caroline Richard, Ben Willing, Diana Mager, Carla Prado), but most leads in actual output are missing from expected output, failing step 1. Additionally, the quality and detail of leads in actual output are generally less comprehensive and do not match the richer background and headline information in expected output, indicating poor alignment with step 2., error: None)

For test case:

  - input: 
    Find me as many leads as possible for the following query:

    Who: professors
    What is the field of study: Human Nutrition
    Where are they located: Department of Agricultural, Food & Nutritional Science at University of Alberta
    Addit

✓ Tests finished 🎉! Run 'deepeval view' to analyze, debug, and save evaluation results on Confident AI.

### Observations:

- [ ] Models allucinate a lot of information on the lack more details (example: sees a name on a article but can't find the person inforamtion, it then allucinate their contact info)
- [ ] The search paths are very narrow and and can totally get out of path when faced with some timeout or other types of page retrieval errors.
- [ ] Even though many parallel calls are made the explorations paths are still not enough to cover possibilities.
- [ ] There is an important need to have a fact check phase. The model tends to ignore important filtering cases like WHERE and CONTEXT clausese. This normally happens when models expand []search and then the filters get "lost" in their context or attention
